In [1]:
import numpy as np
import cv2
import re
import matplotlib
import shutil
import os
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import os
import requests
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)



def download_image(url, folder_name, num):

    # write image to file
    reponse = requests.get(url)
    if reponse.status_code == 200:
        with open(os.path.join(folder_name, str(num)+".jpg"), 'wb') as file:
            file.write(reponse.content)

main_folder_name = 'Images'
if not os.path.isdir(main_folder_name):
    os.makedirs(main_folder_name)

players = ["lionel_messi", "cristiano_ronaldo", "charles_leclerc", "alex_morgan"]
for j in range(len(players)):
    #creating player directory
    sub_folder_name = players[j]
    if not os.path.isdir(os.path.join(main_folder_name, sub_folder_name)):
        os.makedirs(os.path.join(main_folder_name, sub_folder_name))
    player = players[j].split("_")  # #creating search string
    search_string = player[0]+"+"+player[1]+"+"+"face"

    url = f"https://www.google.com/search?q={search_string}&source=lnms&tbm=isch"
    driver.get(url)
    time.sleep(7)
    for i in range(1, 55):
        img_url = ""
        #At div 25 we do not have images.
        if i % 25 != 0:
            try:
                image_container = driver.find_element(By.XPATH,f'//*[@id="islrg"]/div[1]/div[{i}]/a[1]/div[1]/img').click()
                time.sleep(3)
                img_url = driver.find_element(By.XPATH,'//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div[3]/div[1]/a/img[1]').get_attribute("src")
                print(img_url)
                time.sleep(1)
            except:
                print("Could not click")
            try:
                if img_url !="":
                    download_image(img_url, os.path.join(main_folder_name, sub_folder_name), i)
            except:
                print("Could not download.")

In [ ]:
face_cascade = cv2.CascadeClassifier('opencv/haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('opencv/haarcascades/haarcascade_eye.xml')

In [ ]:
scrapped_data = "Images/"
dataset_path = "cropped_dataset/"

In [ ]:
dataset_images = []
names = {}
cropped_image_dirs = []

if os.path.exists(dataset_path):
     shutil.rmtree(dataset_path)         #delete current contents of the folder if folder exists

for img_dir in img_dirs:
    name = img_dir.split('/')[-1]        #get celebrity name from it directory
    print(name)
    names[name] = []
    c=1
    for entry in os.scandir(img_dir):
        path = entry.path
        path_str = re.sub(r'\\', '/', path)        #replace double backslash with frontslash to get image path
        img = cv2.imread(path_str)
        print(path_str)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)   #convert to greyscale
        faces_detected = face_cascade.detectMultiScale(gray, 1.3, 5)  #detect faces
        for (x,y,w,h) in faces_detected:
            gray_img_face = gray[y:y+h, x:x+w]                        #crop face in greyscale and colored image
            colored_face = img[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(gray_img_face)        #detect eyes to ensure image is clear
            if len(eyes) >= 2:                                        #only add image in dataset if full face with eyes visib
                new_dataset = dataset_path + name
                if not os.path.exists(new_dataset):
                    os.makedirs(new_dataset)
                    cropped_image_dirs.append(new_dataset)
                cropped_face = name + str(c) + ".jpg"
                file_path = new_dataset + "/" + cropped_face
                
                cv2.imwrite(file_path, colored_face)
                names[name].append(file_path)
                c += 1

In [ ]:
import numpy as np
from random import randint
import random

# Define the input and output directories
input_dir = "cropped_dataset/"


rotation_angles = [90, -90, 180]

for subfolder in os.listdir(input_dir):
    name = subfolder.split('/')[-1]        #get celebrity name from it directory
    print(name)
    subfolder_path = os.path.join(input_dir, subfolder)
    print(subfolder_path)
  
    for filename in os.listdir(subfolder_path):
    
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(subfolder_path, filename)
            image = cv2.imread(image_path)
            
        angle = random.choice(rotation_angles)  # Applying augmentation
        if angle == 90:
            rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        elif angle == -90:
            rotated_image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
        elif angle == 180:
             rotated_image = cv2.rotate(image, cv2.ROTATE_180)
       
        output_filename = f"{os.path.splitext(filename)[0]}_aug_.jpg"
        output_path = os.path.join(subfolder_path, output_filename)
        names[name].append(output_path)
        cv2.imwrite(output_path, rotated_image)
        